In [37]:
import glob
import os
import subprocess

def convert(n, d):
    get_ipython().system("jupyter nbconvert {} --to markdown --output {}".format(n, d))
    #subprocess.check_call(["jupyter", "nbconvert", n, "--to", "markdown", "--output", d])

highlevel_dirs = (
 #   ["../tools"] + 
    sorted(glob.glob("../sem16*"))
)

print("Highlevel dirs:", highlevel_dirs)

Highlevel dirs: ['../sem16-fcntl-dup-pipe']


In [38]:
for subdir in highlevel_dirs:
    notebooks = glob.glob(subdir + "/*.ipynb")
    print(subdir, notebooks)
    for m in glob.glob(subdir + "/*.md"):
        os.remove(m)
    if len(notebooks) == 1:
        convert(notebooks[0], "README")
    else:
        for n in notebooks:
            convert(n, os.path.basename(n.replace(".ipynb", "")))
        

../sem16-fcntl-dup-pipe ['../sem16-fcntl-dup-pipe/fcntl-dup-pipe.ipynb']
[NbConvertApp] Converting notebook ../sem16-fcntl-dup-pipe/fcntl-dup-pipe.ipynb to markdown
[NbConvertApp] Writing 106846 bytes to ../sem16-fcntl-dup-pipe/README.md


In [39]:
import re

def basic_improve(fname):
    with open(fname, "r") as f:
        r = f.read()
    for b in ["\x00", "\x1B", "\x08"]:
        r = r.replace(b, "")
    with open(fname, "w") as f:
        f.write(r)
    get_ipython().system("dos2unix {}".format(fname))

def improve_md(fname):
    with open(fname, "r") as f:
        r = f.read()
    r = r.replace("```python\n%%cpp", "```cpp\n%%cpp")
    r = r.replace('\n', "SUPER_SLASH" + "_N_REPLACER")
    r = re.sub(r'\<\!--MD_BEGIN_FILTER--\>.*?\<\!--MD_END_FILTER--\>', "", r)
    r = re.sub(r'(\#SET_UP_MAGIC_BEGIN.*?\#SET_UP_MAGIC_END)', "<too much code>", r)
    r = r.replace("SUPER_SLASH" + "_N_REPLACER", '\n')
    
    def file_repl(matchobj, path=os.path.dirname(fname)):
        fname = os.path.join(path, matchobj.group(1))
        if fname.find("__FILE__") == -1:
            with open(fname, "r") as f:
                return "\n```\n" + f.read() + "\n```\n"
    
    r = r.replace("</td>", "")
    r = r.replace("</tr>", "")
    
    r = re.sub(r'\<\!--MD_FROM_FILE (.*?) --\>', file_repl, r)
    with open(fname, "w") as f:
        f.write(r)
        
def improve_file(fname):
    basic_improve(fname)
    if fname.endswith(".md"):
        improve_md(fname)


In [40]:
for sfx in [".ipynb", ".md"]:
    for hdir in highlevel_dirs:
        for fname in glob.glob("./{}/*".format(hdir) + sfx):
            improve_file(fname)

dos2unix: converting file ./../sem16-fcntl-dup-pipe/fcntl-dup-pipe.ipynb to Unix format ...
dos2unix: converting file ./../sem16-fcntl-dup-pipe/README.md to Unix format ...


In [41]:
add_cmd = "git add --ignore-errors "
add_cmd_f = "git add"
for subdir in highlevel_dirs:
    for sfx in [".ipynb", ".md", ".c", ".cpp"]:
        add_cmd += " {}/*{}".format(subdir, sfx)
    add_cmd_f += " -f {}/bash_popen_tmp/*.html".format(subdir)
    
def execute_cmd(cmd):
    print(">", cmd)
    get_ipython().system(cmd)
    
execute_cmd(add_cmd)
execute_cmd(add_cmd_f)
execute_cmd("git add -u")
execute_cmd("git commit -m 'yet another update'")
execute_cmd("git push origin master")

> git add --ignore-errors  ../sem16-fcntl-dup-pipe/*.ipynb ../sem16-fcntl-dup-pipe/*.md ../sem16-fcntl-dup-pipe/*.c ../sem16-fcntl-dup-pipe/*.cpp
fatal: pathspec '../sem16-fcntl-dup-pipe/*.c' did not match any files
> git add -f ../sem16-fcntl-dup-pipe/bash_popen_tmp/*.html
fatal: pathspec '../sem16-fcntl-dup-pipe/bash_popen_tmp/*.html' did not match any files
> git add -u
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
> git commit -m 'yet another update'
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
[master f122ce0] yet another update
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
 2 files changed, 2 insertions(+), 2 deletions(-)
> git push origin master
Counting objects: 5, done.
Compressing objec